# Import aller nötigen Module

In [ ]:
import os

import numpy as np
import pandas as pd
#Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# Filtering
from scipy.signal import butter, filtfilt
from scipy import stats

# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Hilfsfunktionen
Hier definieren wir unsere Hilfsfunktionen zum Plotten

In [ ]:
#Vordefinierte Hilfsfunktion zum plotten
def plot_filter(dataframe, data_column_name, filtered_column_name, title):
    fig = px.line(dataframe, x='Time (s)', y=[data_column_name, filtered_column_name], title=title)
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text=filtered_column_name)
    fig.show() 

# Datenimport

Hier laden wir unsere .pyphox Dateien und fangen ab, falls diese nicht vorhanden sein solte. Zudem setzen wir schoonmal die Spaltennamen fest um diese im fortlaufenen Programm nutzen zu können

In [ ]:
file_name = "RawDataWisch.csv"

# Error-Handling
try:
    dataframe = pd.read_csv("RawDataWisch.csv")
except FileNotFoundError:
    print(f"Error: File not found")
    exit()


# Spaltennamen festsetzen: 
time_col = "Time (s)"
gx_col = "Linear Acceleration x (m/s^2)"
gy_col = "Linear Acceleration y (m/s^2)"
gz_col = "Linear Acceleration z (m/s^2)"
abs_col = "Absolute acceleration (m/s^2)"

# Preview of Data
print("Dataset:")
print(dataframe.head())

# Frequenzfilter

Hier wird das Rauschen reduziert. Dafür nutzen wir Butterworth aus der Vorlesung. Damit werden hoch- bzw. niederfrequente Signale gefiltert. 
Zuerst folgen die niederfrequenten Signale, dann die hochfrequenten:

In [ ]:
filtered_dataframe = dataframe.copy()

filter_order = 4  

low_cutoff = 0.5
high_cutoff = 5 

# Hier nutzen wir den butterwort filter um die Daten zu glätten
b, a = butter(filter_order, [low_cutoff, high_cutoff], btype='band', analog=False, fs =200)

# Apply the filter to acceleration signals
filtered_dataframe['filtered_X_band'] = filtfilt(b, a, dataframe['Linear Acceleration x (m/s^2)'])
filtered_dataframe['filtered_Y_band'] = filtfilt(b, a, dataframe['Linear Acceleration y (m/s^2)'])
filtered_dataframe['filtered_Z_band'] = filtfilt(b, a, dataframe['Linear Acceleration z (m/s^2)'])

plot_filter(filtered_dataframe, 'Linear Acceleration x (m/s^2)', 'filtered_X_band', 'Acceleration X with Band-Pass Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration y (m/s^2)', 'filtered_Y_band', 'Acceleration Y with Band-Pass Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration z (m/s^2)', 'filtered_Z_band', 'Acceleration Z with Band-Pass Filtering')

# Ausreißer Bereinigung
Hier wollen wir nun die möglichen Ausreißer bereinigen: 

In [ ]:
# Berechnung der Z-Scores für jede Spalte (X, Y, Z)
filtered_dataframe['Z_Score_X'] = stats.zscore(filtered_dataframe['Linear Acceleration x (m/s^2)'])
filtered_dataframe['Z_Score_Y'] = stats.zscore(filtered_dataframe['Linear Acceleration y (m/s^2)'])
filtered_dataframe['Z_Score_Z'] = stats.zscore(filtered_dataframe['Linear Acceleration z (m/s^2)'])

# Festlegen eines Schwellenwerts für Z-Scores, ab dem ein Punkt als Ausreißer betrachtet wird
z_score_threshold = 4  

# Filtern der Ausreißer aus dem filterd_dataframe (nur wenn alle drei Achsen unterhalb des Threshold liegen sind Daten ok)
filtered_dataframe = filtered_dataframe[(abs(filtered_dataframe['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframe['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframe['Z_Score_Z']) <= z_score_threshold)]



# Jetzt enthält 'filtered_dataframe' nur noch die Datenpunkte, die nicht als Ausreißer betrachtet werden.
print(dataframe.shape)
print(filtered_dataframe.shape)

#Plotting
plot_filter(filtered_dataframe, 'Linear Acceleration x (m/s^2)', 'Z_Score_X', 'Acceleration X with Z-Score Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration y (m/s^2)', 'Z_Score_Y', 'Acceleration Y with Z-Score Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration z (m/s^2)', 'Z_Score_Z', 'Acceleration Z with Z-Score Filtering')


# Windowing
Hier nutzen wir Windowing um die Daten zu zerlegen. Das brauchen wir später für die Features-Berechnung

In [ ]:
# Da fs = 200 Hz setzen wir das Fenster auf 0.5 Sekunden um auch kurze Bewegungsereignisse abzubilden. 
# Wir könnten window_size aber auch auf 200 setzen.
window_size = 100 
overlap = 50 

#Liste initiieren für Fenster
windows = []

#Für jedes Fenster in den Daten neuen Dataframe erstellen, der in Liste abgelegt wird 
for i in range(0, len(filtered_dataframe), window_size - overlap):
    window = filtered_dataframe.iloc[i:i+window_size]
    windows.append(window)

#Ersten Dataframe in der Liste ausgeben lassen 
print(windows[0].head())
#Anzahl Datenpunkte des ersten Dataframes ausgeben lassen 
print(len(windows[0]))
#Spalten des ersten Dataframes ausgeben lassen 
print(windows[0].columns)

#Ausgeben lassen wie viele Dataframes also Fenster entstanden sind 
print (len(windows))


# Features erstellen
Hier berechnen wir die Merkmale, die wesentliche Eigneschaften der Daten kompakt beschreiben. Diese dienen als Eingabe für den späteren Entscheidungsbaum. 

In [ ]:
#Leere Liste erstellen, um Features zu speichern 
feature_list = []

#Jeden Dataframe in Windows einzeln durchgehen und für jede der drei Achsen die Features berechnen 
for window in windows:
    acceleration_data = window[['Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)']]
    #Mittelwert
    acceleration_mean = acceleration_data.mean()
    #Standardabweichung
    acceleration_std_dev = acceleration_data.std()
    #Minimum
    acceleration_min = acceleration_data.min()
    #Maximum
    acceleration_max = acceleration_data.max()
    
    #Berechnete Werte an Liste anhängen, unter entsprechend benannten Spalten 
    feature_list.append({
        'BeschleunigungX_Mittelwert': acceleration_mean.iloc[0],
        'BeschleunigungY_Mittelwert': acceleration_mean.iloc[1],
        'BeschleunigungZ_Mittelwert': acceleration_mean.iloc[2],
        'BeschleunigungX_Standardabweichung': acceleration_std_dev.iloc[0],
        'BeschleunigungY_Standardabweichung': acceleration_std_dev.iloc[1],
        'BeschleunigungZ_Standardabweichung': acceleration_std_dev.iloc[2],
        'BeschleunigungX_Minimum': acceleration_min.iloc[0],
        'BeschleunigungY_Minimum': acceleration_min.iloc[1],
        'BeschleunigungZ_Minimum': acceleration_min.iloc[2],
        'BeschleunigungX_Maximum': acceleration_max.iloc[0],
        'BeschleunigungY_Maximum': acceleration_max.iloc[1],
        'BeschleunigungZ_Maximum': acceleration_max.iloc[2]
    })

#In einen Pandas Dataframe konvertieren und ausgeben
feature_df = pd.DataFrame(feature_list)

print(feature_df.head())

#Dimensionen der Feautures ausgeben lassen 
feature_df.shape


## Entscheidungsbaum

Hier trainieren wir den Entscheidungsbaum

In [ ]:
# Label für Datensatz setzen 
feature_df["label"] = "Wischen" # Das ändern wenn anderer Datensatz

# Features und Zielvariable trennen
X = feature_df.drop(columns=["label"])
Y = feature_df["label"]

# Train/Test-Split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

#Entscheidungsbaum(CART)

clf = DecisionTreeClassifier(
    criterion="gini", 
    max_depth=5,
    min_samples_leaf=5,
    random_state=42
)

#Training
clf.fit(X_train, y_train)

#Validierung
y_pred = clf.predict(X_test)

print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion Matrix", confusion_matrix(y_test, y_pred))
print("Classification Report" + classification_report(y_test, y_pred))


# Entscheidungsbaum speichern

import joblib

joblib.dump(clf, "decision_tree_model.pkl")
print("Modell gespeichert")




## Entscheidungsbaum laden und neue Daten klassifizieren

Hier laden wir den trainierten Entscheidungsbaum und nutzen diesen zu Klassifikation von neuen Sensordaten

In [ ]:
# Modell laden
model = joblib.load("decision_tree_model.pkl")

# Beispieö: erstes Feature-Fenster klassifizieren 
sample = X.iloc[[0]]
prediction = model.predict(sample)

print("Vorhergesagte Klasse:", prediction[0])




## Dobot Aktionen anahnd der Vorhersage ausführen

Hier verknüpfen wir die prediction mit dem Bot

In [ ]:
from dobot import Dobot

bot = Dobot("COM 3") # Je nachdem auf welchem Bot wir arbeiten

def wischen(bot):
    print("Aktion: Wischen")
    x_min = 220
    x_haken = 260
    x_max = 300

    y_min = -80
    y_haken = -40
    y_max = -20

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_haken, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Kastenstartposition')
    bot.move_to_to(x_max, y_min, z_max, 0)

    print('Kastenposition 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kastenposition 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kastenposition 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kastenposition 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kastenposition 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Hakenstartposition')
    bot.move_to(x_haken, y_max, z_max, 0)

    print('Position 1')
    bot.move_to(x_haken, y_max, z_min, 0)

    print('Position 2')
    bot.slide_to(x_min, y_haken, z_min, 0)

    print('Position 3')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Zurück nach Position 1')
    bot.move_to(x_max, y_min, z_max, 0)

def hammer(bot):

    print("Aktion: Hammer")
    x_min = 220
    x_max = 300

    y_min = 20
    y_max = 80

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_max, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Kreuz-Position 1')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Hochfahren 2')
    bot.move_to(x_min, y_max, z_max, 0)

    print('Startposition 2')
    bot.move_to(x_max, y_max, z_max, 0)

    print('Kreuz-Position 2')
    bot.move_to(x_max, y_max, z_min, 0)

    print('Kreuz-Position 3')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Hochfahren 3')
    bot.move_to(x_min, y_min, z_max, 0)